In [ ]:
!pip install ultralytics
!pip install torch

In [ ]:
import torch
import cv2
import os

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}, {torch.cuda.get_device_name(torch.cuda.current_device())}')

In [ ]:
from ultralytics import YOLO

model = YOLO("yolov9c.pt")
model.info()

results = model.train(data="/content/dataset/data.yaml", epochs=100, imgsz=416, batch=64, workers=16)

In [ ]:
model.save("/content/trained_yolov9c.pt")
results = model("input.mp4")

In [ ]:
def predict_and_detect(model, img, conf=0.5, rectangle_thickness=2, text_thickness=2):
    results = model.predict(source=img, conf=conf)  # Use the YOLO model to predict
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), rectangle_thickness)
            text = f"License Plate: {box.conf[0].item():.2f}"
            cv2.putText(img, text, (x1, y1 - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), text_thickness)
    return img

In [ ]:
def create_video_writer(video_cap, output_filename):
    frame_width = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    writer = cv2.VideoWriter(output_filename, fourcc, fps, (frame_width, frame_height))

    return writer

In [ ]:
def process_video(input_video_path, output_filename, model, conf=0.5):
    output_dir = "/content/outputs"
    os.makedirs(output_dir, exist_ok=True)

    output_video_path = os.path.join(output_dir, output_filename)

    cap = cv2.VideoCapture(input_video_path)

    if not cap.isOpened():
        print("Error: Unable to open file.")
        return

    writer = create_video_writer(cap, output_video_path)

    frame_count = 0
    while True:
        success, img = cap.read()
        if not success:
            print("Error: Unable to read frame or video has ended.")
            break

        result_img = predict_and_detect(model, img, conf=conf)

        writer.write(result_img)
        cv2.waitKey(1)

        frame_count += 1
        print(f"Frame {frame_count} processed.")

    cap.release()
    writer.release()
    cv2.destroyAllWindows()
    print(f"The video has been processed successfully and saved to {output_video_path}.")

In [ ]:
# first run with 0.5 confidence

input_video_path = "input.mp4"
output_video_path = "output-0.5confidence.mp4"
process_video(input_video_path, output_video_path, model, conf=0.5)

In [ ]:
# second run with 0.1 confidence

input_video_path = "input.mp4"
output_video_path = "output-0.1confidence.mp4"
process_video(input_video_path, output_video_path, model, conf=0.1)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install easyocr

In [ ]:
import easyocr
import numpy as np

def detect_color(roi):

  hsv = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
  colors = {
        "Red": ([0, 120, 70], [10, 255, 255]),
        "Green": ([36, 25, 25], [70, 255, 255]),
        "Blue": ([94, 80, 2], [126, 255, 255]),
        "Yellow": ([15, 150, 150], [35, 255, 255]),
        "Black": ([0, 0, 0], [180, 255, 30]),
        "White": ([0, 0, 200], [180, 20, 255]),
        "Gray": ([0, 0, 40], [180, 20, 200])
    }
  max_pixels = 0
  dominant_color = "Unknown"

  for color, (lower, upper) in colors.items():
      mask = cv2.inRange(hsv, np.array(lower), np.array(upper))
      num_pixels = cv2.countNonZero(mask)

      if num_pixels > max_pixels:
          max_pixels = num_pixels
          dominant_color = color

  return dominant_color

In [ ]:
# update predict_and_detect to use EasyOCR and color detection
reader = easyocr.Reader(['en'])

def predict_and_detect(model, img, conf=0.5, rectangle_thickness=2, text_thickness=2):
    results = model.predict(source=img, conf=conf)  # Use the YOLO model to predict
    for result in results:
        for box in result.boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            plate_img = img[y1:y2, x1:x2]

            expand_y = int((y2 - y1) * 4)
            expand_x = int((x2 - x1) * 3)
            y1_expanded = max(y1 - expand_y // 2, 0)
            y2_expanded = min(y2 + expand_y // 2, img.shape[0])
            x1_expanded = max(x1 - expand_x // 2, 0)
            x2_expanded = min(x2 + expand_x // 2, img.shape[1])

            car_roi = img[y1_expanded:y2_expanded, x1_expanded:x2_expanded]

            ocr_result = reader.readtext(plate_img)
            license_plate_text = "Unknown"
            if ocr_result:
                license_plate_text = ocr_result[0][-2]

            car_color = detect_color(car_roi)

            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 0, 255), rectangle_thickness)
            text = f"{license_plate_text}, {car_color}"
            cv2.putText(img, text, (x1, y1 - 10), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), text_thickness)
    return img

In [ ]:
# run again with easyocr and color detection

input_video_path = "input.mp4"
output_video_path = "output-final.mp4"
process_video(input_video_path, output_video_path, model, conf=0.5)